# OpenAI Introduction

Use this for [reference](https://platform.openai.com/docs/api-reference/authentication)

In [1]:
%pipenv install openai python-dotenv -q

Note: you may need to restart the kernel to use updated packages.


> Create an OpenAI account and get an API_KEY

### OpenAI:

1. Chat completion API
2. Implement structured output
3. Implement tool calling
4. Implement streaming
5. Implement assistant
6. Response API
7. Chaining of response

In [2]:
from openai import OpenAI
from os import getenv
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
client = OpenAI(
  api_key=getenv("OPENAI_API_KEY")
)

### List Models

In [14]:
models = client.models.list()
model_names = [model.id for model in models if "gpt" in model.id]

In [15]:
model_names

['gpt-4o-audio-preview-2024-10-01',
 'gpt-4o-mini-audio-preview',
 'gpt-4o-audio-preview',
 'gpt-4.1-nano',
 'gpt-3.5-turbo-instruct-0914',
 'gpt-4o-mini-search-preview',
 'gpt-4.1-nano-2025-04-14',
 'gpt-3.5-turbo-16k',
 'gpt-3.5-turbo-1106',
 'gpt-4o-search-preview',
 'gpt-3.5-turbo-instruct',
 'gpt-3.5-turbo',
 'gpt-4o-mini-search-preview-2025-03-11',
 'gpt-4o-2024-11-20',
 'gpt-4o-2024-05-13',
 'gpt-4o-mini-tts',
 'gpt-4o-transcribe',
 'gpt-4.5-preview',
 'gpt-4.5-preview-2025-02-27',
 'gpt-4o-search-preview-2025-03-11',
 'gpt-image-1',
 'gpt-4o',
 'gpt-4o-2024-08-06',
 'gpt-4o-mini-2024-07-18',
 'gpt-4.1-mini',
 'gpt-4o-mini',
 'gpt-4o-mini-audio-preview-2024-12-17',
 'gpt-3.5-turbo-0125',
 'gpt-4o-mini-transcribe',
 'gpt-4.1-mini-2025-04-14',
 'gpt-4.1',
 'gpt-4.1-2025-04-14']

| 1. Chat Completion |
|--|


The Chat Completions API endpoint will generate a model response from a list of messages comprising a conversation.

In [21]:
MODEL_NAME = "gpt-3.5-turbo"

In [ ]:
completion = client.chat.completions.create(
  model=MODEL_NAME,
  store=False,
  messages=[
    {"role": "user", "content": "write a haiku about ai"}
  ]
)

In [ ]:
print(completion.choices[0].message.content)

| 2. Structured Output |
|--|

Structured Outputs is a feature that ensures the model will always generate responses that adhere to your supplied JSON Schema,

In [ ]:
!pipenv install pydantic

In [19]:
from pydantic import BaseModel

In [20]:
class UserRegistration(BaseModel):
    name: str
    age: int
    tier: int

In [ ]:
completion = client.beta.chat.completions.parse(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": "Extract the user information."},
        {"role": "user", "content": "We have a new user named Tim aged 25 with a tier 3 account."},
    ],
    response_format=UserRegistration,
)

In [ ]:
event = completion.choices[0].message.parsed
event

| 3. Tool/Function Calling |
|--|

Function calling provides a powerful and flexible way for OpenAI models to interface with your code or external services. 



In [23]:
tools = [{
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Get current temperature for a given location.",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "City and country e.g. Bogotá, Colombia"
                }
            },
            "required": [
                "location"
            ],
            "additionalProperties": False
        },
        "strict": True
    }
}]

| Function -> Tool JSON Convertor |
|--|

Automate tool JSON creation

In [43]:
import inspect
from typing import Callable, get_type_hints

In [55]:
def generate_json_schema(func: Callable) -> str:
    """
    Generate a JSON schema for a given function based on its docstring and type hints.

    Args:
        func (Callable): The Python function to generate a schema for.

    Returns:
        str: A JSON string representing the function's schema.

    Raises:
        ValueError: If the function lacks a docstring.
    """
    # Get function metadata
    if not func.__doc__:
        raise ValueError("Function must have a docstring")

    signature = inspect.signature(func)
    type_hints = get_type_hints(func)

    # Parse docstring
    docstring = inspect.getdoc(func) or ""
    doc_lines = docstring.split('\n')
    description = doc_lines[0].strip()  # First line is the main description
    param_descriptions = {}
    in_args_section = False

    # Extract parameter descriptions
    for line in doc_lines:
        line = line.strip()
        if line.lower().startswith('args:'):
            in_args_section = True
            continue
        if in_args_section and line and ':' in line:
            # Handle lines like "param_name (type): description"
            parts = line.split(':', 1)
            param_part = parts[0].strip()
            desc = parts[1].strip() if len(parts) > 1 else ""
            # Extract param_name from "param_name (type)"
            if '(' in param_part and ')' in param_part:
                param_name = param_part[:param_part.index('(')].strip()
            else:
                param_name = param_part
            if param_name:
                param_descriptions[param_name] = desc

    # Map Python types to JSON schema types
    type_mapping = {
        'str': 'string',
        'int': 'integer',
        'float': 'number',
        'bool': 'boolean',
        'dict': 'object',
        'list': 'array'
    }

    # Build schema
    schema = {
        "type": "function",
        "function": {
            "name": func.__name__,
            "description": description,
            "parameters": {
                "type": "object",
                "properties": {},
                "required": [],
                "additionalProperties": False
            },
            "strict": True
        }
    }

    # Add parameters to schema
    for param_name, param in signature.parameters.items():
        # Get type from type hints, default to str if not specified
        param_type = type_hints.get(param_name, str).__name__
        schema_type = type_mapping.get(param_type, 'string')  # Default to string
        
        schema["function"]["parameters"]["properties"][param_name] = {
            "type": schema_type,
            "description": param_descriptions.get(param_name, f"{param_name} parameter")
        }
        # Mark as required if no default value
        if param.default == inspect.Parameter.empty and param.kind != inspect.Parameter.VAR_KEYWORD:
            schema["function"]["parameters"]["required"].append(param_name)

    return schema

#### Weather function

In [56]:
def get_weather(location: str) -> dict:
    """
    Get current temperature for a given location.

    Args:
        location (str): City and country e.g. Bogotá, Colombia

    Returns:
        dict: A dictionary containing the current temperature and other weather details.
    """
    return {"location": location, "temperature": 25, "unit": "Celsius"}

In [57]:
generate_json_schema(func=get_weather)

{'type': 'function',
 'function': {'name': 'get_weather',
  'description': 'Get current temperature for a given location.',
  'parameters': {'type': 'object',
   'properties': {'location': {'type': 'string',
     'description': 'City and country e.g. Bogotá, Colombia'}},
   'required': ['location'],
   'additionalProperties': False},
  'strict': True}}

In [58]:
tools[0]

{'type': 'function',
 'function': {'name': 'get_weather',
  'description': 'Get current temperature for a given location.',
  'parameters': {'type': 'object',
   'properties': {'location': {'type': 'string',
     'description': 'City and country e.g. Bogotá, Colombia'}},
   'required': ['location'],
   'additionalProperties': False},
  'strict': True}}

#### Tool Calling

In [ ]:
completion = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[{"role": "user", "content": "What is the weather like in Paris today?"}],
    tools=tools
)

In [ ]:
print(completion.choices[0].message.tool_calls)